In [34]:
import os
import json
import base64
from PIL import Image
from io import BytesIO

def convert_labelme_to_yolo(json_dir, output_dir, class_mapping):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_path = os.path.join(json_dir, filename)
            with open(json_path, 'r') as f:
                data = json.load(f)

            image_data = data['imageData']
            image_height = data['imageHeight']
            image_width = data['imageWidth']
            
            shapes = data['shapes']
            yolo_labels = []

            for shape in shapes:
                label_name = shape['label']
                if label_name not in class_mapping:
                    continue

                label = class_mapping[label_name]
                
                if shape['shape_type'] == 'rectangle':
                    points = shape['points']
                    x_min, y_min = points[0]
                    x_max, y_max = points[1]
                    
                    # Convert to YOLO format
                    x_center = (x_min + x_max) / 2.0 / image_width
                    y_center = (y_min + y_max) / 2.0 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

                elif shape['shape_type'] == 'polygon':
                    points = shape['points']
                    x_min = min([point[0] for point in points])
                    x_max = max([point[0] for point in points])
                    y_min = min([point[1] for point in points])
                    y_max = max([point[1] for point in points])

                    # Convert to YOLO format
                    x_center = (x_min + x_max) / 2.0 / image_width
                    y_center = (y_min + y_max) / 2.0 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # Write YOLO label file
            filename_stem = os.path.splitext(filename)[0]
            yolo_path = os.path.join(output_dir, filename_stem + '.txt')
            with open(yolo_path, 'w') as f:
                f.write('\n'.join(yolo_labels))

            # Decode base64 image data and save as image file
            image_data = base64.b64decode(image_data)
            image = Image.open(BytesIO(image_data))
            image_output_path = os.path.join(output_dir, filename_stem + '.jpg')
            image.save(image_output_path)

# Usage
data_rel_path = '../../data'.replace('/', os.path.sep)
data_abs_path = os.path.abspath(data_rel_path)
json_dir = os.path.join(data_abs_path, 'raw_jsons')  # Replace with the path to your JSON files directory
output_dir = os.path.join(data_abs_path, 'dataset')    # Replace with the path to your output directory

# Define class mapping
class_mapping = {
    "arch": 0,
    "vomer": 1
}

convert_labelme_to_yolo(json_dir, output_dir, class_mapping)

In [43]:
# Validate the txt and jpg datasets have identical names. Later copy them into train and val folders

import os
import shutil
from sklearn.model_selection import train_test_split

def organize_dataset(dataset_dir, output_dir, split_ratio=0.2):
    # Check if output directories exist, if not, create them
    images_train_dir = os.path.join(output_dir, 'images/train')
    images_val_dir = os.path.join(output_dir, 'images/val')
    labels_train_dir = os.path.join(output_dir, 'labels/train')
    labels_val_dir = os.path.join(output_dir, 'labels/val')

    for dir_path in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # Get lists of all jpg and txt files
    jpg_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]
    txt_files = [f for f in os.listdir(dataset_dir) if f.endswith('.txt')]

    # Check for 1-to-1 correspondence
    jpg_file_names = set(os.path.splitext(f)[0] for f in jpg_files)
    txt_file_names = set(os.path.splitext(f)[0] for f in txt_files)
    common_file_names = jpg_file_names.intersection(txt_file_names)

    if not (jpg_file_names == txt_file_names):
        print("Warning: Some jpg and txt files do not have corresponding pairs.")

    # Prepare paired file paths
    jpg_files = [os.path.join(dataset_dir, f"{name}.jpg") for name in common_file_names]
    txt_files = [os.path.join(dataset_dir, f"{name}.txt") for name in common_file_names]

    # Split data into training and validation sets
    jpg_train, jpg_val, txt_train, txt_val = train_test_split(
        jpg_files, txt_files, test_size=split_ratio, random_state=42
    )

    # Copy files to their respective directories
    def copy_files(files, output_dir):
        for file in files:
            shutil.copy(file, output_dir)

    copy_files(jpg_train, images_train_dir)
    copy_files(jpg_val, images_val_dir)
    copy_files(txt_train, labels_train_dir)
    copy_files(txt_val, labels_val_dir)

    print(f"Organized dataset into {images_train_dir}, {images_val_dir}, {labels_train_dir}, and {labels_val_dir}")

# Usage
data_rel_path = '../../data/dataset'.replace('/', os.path.sep)
data_abs_path = os.path.abspath(data_rel_path)
output_dir = os.path.join(data_abs_path, 'detect_arch')    # Replace with the path to your output directory

organize_dataset(data_abs_path, output_dir)

Organized dataset into d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\images/train, d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\images/val, d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels/train, and d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels/val


In [2]:
import os
print(os.environ['CONDA_PREFIX'])
try:
    import sklearn
    print("scikit-learn is installed.")
    print("scikit-learn version:", sklearn.__version__)
except ImportError:
    print("scikit-learn is not installed.")

C:\Users\DYPro\.conda\envs\mesh_seg_net
scikit-learn is installed.
scikit-learn version: 1.5.0


In [5]:
!conda install scikit-learn -y

Channels:
 - conda-forge
 - defaults
 - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
 - nvidia
 - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
 - pytorch
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [1]:
import sys
print(sys.executable)

C:\Users\DYPro\.conda\envs\mesh_seg_net\python.exe


In [44]:
# Train Yolov10 model for the first time
import os
import torch
from ultralytics import YOLOv10

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Create a new YOLO model from scratch
model = YOLOv10('yolov10n.yaml')

# Ensure the model uses GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("using device: ", device)
model.to(device)

print(os.getcwd())
# Train the model using the 'arch_detection.yaml' dataset for 50 epochs
results = model.train(data='detect_arch.yaml', epochs=200, device=device)


using device:  cuda
d:\Code\CBCT_Reconstruction_ICP\src\main program
New https://pypi.org/project/ultralytics/8.2.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov10n.yaml, data=detect_arch.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cuda, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\train... 183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 183/183 [00:00<00:00, 1878.62it/s]

train: New cache created: D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\train.cache



val: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\val... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<00:00, 1152.93it/s]

val: New cache created: D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\val.cache


...

    200/200       3.2G     0.4774     0.3646       1.19     0.4746     0.4749      1.199          7        640: 100%|██████████| 12/12 [00:01<00:00,  7.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.67it/s]

                   all         46         46      0.938      0.929      0.979      0.882



200 epochs completed in 0.129 hours.
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train23\weights\last.pt, 5.8MB
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train23\weights\best.pt, 5.8MB

Validating D:\Code\CBCT_Reconstruction_ICP\runs\detect\train23\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
YOLOv10n summary (fused): 285 layers, 2695196 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.25it/s]


                   all         46         46      0.975      0.937      0.976      0.883
                  arch         46         26      0.951      0.885      0.958      0.816
                 vomer         46         20          1      0.989      0.995       0.95
Speed: 1.6ms preprocess, 2.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to D:\Code\CBCT_Reconstruction_ICP\runs\detect\train23


In [41]:
# Continue training the model for another 50 epochs

import os
import torch
from ultralytics import YOLOv10

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 创建新的 YOLO 模型实例
model = YOLOv10('yolov10n.yaml')

# 加载之前训练好的模型权重
checkpoint_path = '../runs/detect/train21/weights/best.pt'
model.load(checkpoint_path)

# 确保模型使用 GPU（如果可用）
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("using device: ", device)
model.to(device)

# 使用 'arch_detection.yaml' 数据集继续训练模型，例如再训练 50 个 epochs
results = model.train(data='detect_arch.yaml', epochs=50, device=device)

# 保存继续训练后的模型
model.save('continued_trained_model.pt')

Transferred 493/595 items from pretrained weights
using device:  cuda
New https://pypi.org/project/ultralytics/8.2.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov10n.yaml, data=detect_arch.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cuda, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\train.cache... 204 images, 0 backgrounds, 0 corrupt: 100%|██████████| 204/204 [00:00<?, ?it/s]
val: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels\val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.0005), 107 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22
Starting training for 50 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


       1/50      3.28G      0.922      2.327      1.419     0.8935      6.898      1.429         28        640: 100%|██████████| 13/13 [00:02<00:00,  5.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.88it/s]

                   all         52         52     0.0032      0.952      0.266      0.223


...

      50/50      2.96G     0.5408     0.6572      1.074     0.5685      1.665      1.079         12        640: 100%|██████████| 13/13 [00:01<00:00,  8.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.25it/s]

                   all         52         52      0.927      0.911       0.97      0.846



50 epochs completed in 0.042 hours.
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22\weights\last.pt, 5.8MB
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22\weights\best.pt, 5.8MB

Validating D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
YOLOv10n summary (fused): 285 layers, 2695196 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.35it/s]


                   all         52         52      0.927      0.911       0.97      0.846
                  arch         52         31      0.905      0.917      0.971      0.794
                 vomer         52         21       0.95      0.905      0.969      0.899
Speed: 1.0ms preprocess, 2.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to D:\Code\CBCT_Reconstruction_ICP\runs\detect\train22


In [42]:
import os
from ultralytics import YOLOv10
import cv2
import shutil

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Train the model using the 'coco8.yaml' dataset for 3 epochs
model = YOLOv10('../runs/detect/train21/weights/best.pt')

# 定义输入和输出目录
test_images_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/04/jpg'
output_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/04/jpg_predict'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)

# 获取测试集中的所有图像文件
test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg')]
arch_positive_indices = {}
vomer_positive_indices = {}

def plot_predictions(img, preds, output_path):
    for pred in preds:
        x1, y1, x2, y2 = map(int, pred['bbox'])
        label = f"{pred['label']} {pred['confidence']:.2f}"
        print(label)

        if (pred['label'] == 0):
            color = (0, 255, 0)
        elif (pred['label'] == 1):
            color = (255, 0, 0)
        # 绘制矩形框
        cv2.rectangle(img, (x1, y1), (x2, y2), color=color, thickness=2)

        # 绘制标签
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 保存绘制好预测结果的图像
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

for index, img_name in enumerate(test_images):
    img_path = os.path.join(test_images_dir, img_name)
    img = cv2.imread(img_path)  # 使用 OpenCV 读取图像
    print("detecting", img_path)
    # if img is None:
    #     print(f"Failed to load image {img_path}")
    #     continue  # 跳过无法读取的图像
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 转换为 RGB 格式
    results = model(img_path)  # 对图片进行预测
    preds = []
    has_box = False
    for result in results:
        for box in result.boxes:
            confidence = box.conf[0].item()  # 获取置信度
            if (confidence <= 0.35):
                print('Skipped box with confidence: {} in image {}'.format(confidence, img_name))
                continue
            has_box = True
            bbox = box.xyxy[0].tolist()  # 获取 bbox 的 [x1, y1, x2, y2] 坐标
            label = box.cls[0].item()  # 获取标签
            pred = {
                'bbox': bbox,
                'label': label,
                'confidence': confidence
            }
            preds.append(pred)

    if has_box:
        plot_predictions(img, preds, os.path.join(output_dir, img_name))  # 保存带框的图片
        for pred in preds:
            if pred['label'] == 0:
                current_arch_confidence = arch_positive_indices.get(index + 1, -1)
                if pred['confidence'] > current_arch_confidence:
                    arch_positive_indices[index + 1] = pred['confidence']
            elif pred['label'] == 1:
                current_vomer_confidence = vomer_positive_indices.get(index + 1, -1)
                if pred['confidence'] > current_vomer_confidence:
                    vomer_positive_indices[index + 1] = pred['confidence']


print("所有测试图片的预测结果已保存到", output_dir)

print("arch的索引分别是：")
for key, value in arch_positive_indices.items():
    print(key, value)
print("vomer的索引分别是：")
for key, value in vomer_positive_indices.items():
    print(key, value)

detecting D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0001.jpg

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\04\jpg\04_slice_0001.jpg: 640x640 (no detections), 9.5ms
Speed: 3.0ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
detecting D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0002.jpg

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\04\jpg\04_slice_0002.jpg: 640x640 1 vomer, 8.2ms
Speed: 2.1ms preprocess, 8.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Skipped box with confidence: 0.3342800438404083 in image 04_slice_0002.jpg
detecting D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0003.jpg

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\04\jpg\04_slice_0003.jpg: 640x640 (no detections), 7.1ms
Speed: 2.7ms preprocess, 7.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
detecting D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0004.jpg

image 1/1 D:\Code\CBCT_Reconstruction_ICP

In [2]:
# Adaptive threshold filtering

import os
import cv2

def apply_adaptive_threshold(input_dir, output_dir):
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    # 获取输入目录中的所有图像文件
    image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
    
    for img_name in image_files:
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)
        
        # 读取图像
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)  # 读取为灰度图像
        if image is None:
            print(f"无法读取图像: {input_path}")
            continue
        
        # 应用自适应阈值处理
        adaptive_thresh = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
            cv2.THRESH_BINARY_INV, 11, 5
        )
        
        # 保存处理后的图像
        cv2.imwrite(output_path, adaptive_thresh)
        print(f"已处理并保存图像: {output_path}")

# 定义输入和输出目录
input_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/01/jpg'
output_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg'

# 应用自适应阈值处理
apply_adaptive_threshold(input_dir, output_dir)

print("所有图像处理完毕。")

已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0001.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0002.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0003.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0004.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0005.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0006.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0007.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0008.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0009.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0010.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0011.jpg
已处理并保存图像: D:/Code/CBCT_Reconstru

In [6]:
# Adaptive threshold filtering 并去除短线段与点
import os
import cv2
import numpy as np

def apply_adaptive_threshold_and_filter(input_dir, output_dir):
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    # 获取输入目录中的所有图像文件
    image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
    
    for img_name in image_files:
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)
        
        # 读取图像
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)  # 读取为灰度图像
        if image is None:
            print(f"无法读取图像: {input_path}")
            continue
        
        # 应用自适应阈值处理
        adaptive_thresh = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
            cv2.THRESH_BINARY_INV, 17, 2
        )
        
        # 定义形态学操作的内核
        kernel = np.ones((3, 3), np.uint8)
        
        # 进行形态学开运算去除噪点
        morph_open = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)
        
        # 进行形态学闭运算填充线段间的空隙
        morph_close = cv2.morphologyEx(morph_open, cv2.MORPH_CLOSE, kernel)

        # 连通组件分析去除小的联通组件
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(morph_close, connectivity=8)
        min_area = 200  # 设定最小区域面积阈值
        
        filtered_image = np.zeros_like(morph_close)
        
        for i in range(1, num_labels):  # 从1开始，跳过背景
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                filtered_image[labels == i] = 255
        
        # 保存处理后的图像
        cv2.imwrite(output_path, filtered_image)
        print(f"已处理并保存图像: {output_path}")

# 定义输入和输出目录
base_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/04/'
input_dir = os.path.join(base_dir, 'jpg')
output_dir = os.path.join(base_dir, 'adaptive_threshold_jpg')

# 应用自适应阈值处理和过滤
apply_adaptive_threshold_and_filter(input_dir, output_dir)

print("所有图像处理完毕。")

已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0001.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0002.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0003.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0004.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0005.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0006.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0007.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0008.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0009.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0010.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0011.jpg

['AABBTreePolyline2', 'AABBTreePolyline3', 'AffineXf2d', 'AffineXf2f', 'AffineXf3d', 'AffineXf3f', 'BitSet', 'BooleanOperation', 'BooleanResMapObj', 'BooleanResMaps', 'BooleanResult', 'BooleanResultMapper', 'BoostBitSet', 'Box2d', 'Box2f', 'Box3d', 'Box3f', 'Color', 'Contour2d', 'Contour2f', 'Contour3d', 'Contour3f', 'ContourToDistanceMapParams', 'Contours2d', 'Contours2f', 'Contours3d', 'Contours3f', 'DecimateResult', 'DecimateSettings', 'DecimateStrategy', 'DistanceMap', 'DistanceMapToWorld', 'EdgeBitSet', 'EdgeId', 'EdgeMap', 'EdgeMetric', 'EdgePoint', 'ExpectedVoid', 'FaceBitSet', 'FaceFace', 'FaceHashMap', 'FaceId', 'FaceIncidence', 'FaceMap', 'FaceNormals', 'FillHoleMetric', 'FillHoleParams', 'FillHoleParamsMultipleEdgesResolveMode', 'FindOverhangsSettings', 'FixSelfIntersectionMethod', 'FixSelfIntersectionSettings', 'FloatGrid', 'GeneralOffsetParameters', 'GeneralOffsetParametersMode', 'GeodesicPathApprox', 'GridType', 'HolesVertIds', 'ICP', 'ICPMethod', 'ICPMode', 'ICPProperties', 'InflateSettings', 'IntersectionPrecomputesd', 'IntersectionPrecomputesf', 'ItemsView[FaceHashMap]', 'ItemsView[VertHashMap]', 'ItemsView[WholeEdgeHashMap]', 'KeysView[FaceHashMap]', 'KeysView[VertHashMap]', 'KeysView[WholeEdgeHashMap]', 'LaplacianEdgeWeightsParam', 'Line3d', 'Line3f', 'LineSegm2f', 'LineSegm3f', 'LoadDCMResult', 'LoadDCMResults', 'LoadingTiffSettings', 'Matrix2d', 'Matrix2f', 'Matrix3d', 'Matrix3f', 'Mesh', 'MeshApproxRelaxParams', 'MeshBuilderSettings', 'MeshIntersectionResult', 'MeshLoadSettings', 'MeshPart', 'MeshProjectionResult', 'MeshRelaxParams', 'MeshSignedDistanceResult', 'MeshToDistanceMapParams', 'MeshToDistanceVolumeParams', 'MeshToVolumeParams', 'MeshToVolumeParamsType', 'MeshTopology', 'MeshTriPoint', 'MovementBuildBodyParams', 'NestedComponenetsMode', 'OffsetParameters', 'PackMapping', 'PartMapping', 'Path', 'Plane3f', 'PointCloud', 'PointOnFace', 'PointPair', 'PointPairs', 'PointsToMeshParameters', 'Polyline2', 'Polyline3', 'PolylineTopology', 'RelaxApproxType', 'RelaxParams', 'RemeshSettings', 'SceneObject', 'SegmPointf', 'SharpOffsetParameters', 'SignDetectionMode', 'SignedDistanceToMeshResult', 'SimpleVolume', 'SlicePlane', 'SphereParams', 'StitchHolesParams', 'SubdivideSettings', 'SurfacePath', 'SurfacePaths', 'SymbolMeshParams', 'TextAlignParams', 'ThreeVertIds', 'TriPointf', 'Triangulation', 'TriangulationParameters', 'UndirectedEdgeBitSet', 'UndirectedEdgeId', 'UndirectedEdgeMap', 'UndirectedEdgeUndirectedEdge', 'UniteManyMeshesParams', 'ValuesView[FaceHashMap]', 'ValuesView[VertHashMap]', 'ValuesView[WholeEdgeHashMap]', 'VdbVolume', 'Vector2d', 'Vector2f', 'Vector2i', 'Vector3d', 'Vector3f', 'Vector3i', 'VectorFloatByVert', 'VertBitSet', 'VertColorMap', 'VertCoords', 'VertCoords2', 'VertHashMap', 'VertId', 'VertMap', 'ViewportId', 'ViewportMask', 'VoxelsSaveSavingSettings', 'WholeEdgeHashMap', 'WholeEdgeMap', 'WindingMode', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'addBaseToPlanarMesh', 'alignTextToMesh', 'angle', 'boolean', 'buildBottom', 'buildCylinderBetweenTwoHoles', 'buildShortestPath', 'buildShortestPathBiDir', 'buildSmallestMetricPath', 'buildSmallestMetricPathBiDir', 'calcOrientedArea', 'computeDistanceMapD', 'comnces', 'computeThicknessAtVertices', 'copyMesh', 'createSymbolsMesh', 'cross', 'cutAlongEdgeLoop', 'cutMeshWithPlane', 'decimateMesh', 'detectBasisTunnels', 'detectTunnelFaces', 'dilateRegion', 'dilateRegionByMetric', 'distanceMapTo2DIsoPolyline', 'distanceMapToMesh', 'dot', 'doubleOffsetMesh', 'edgeCurvMetric', 'edgeLengthMetric', 'edgeTableSymMetric', 'erodeRegion', 'erodeRegionByMetric', 'expand', 'extractPlaneSections', 'fillContourLeft', 'fillContourLeftByGraphCut', 'fillHole', 'findCollidingTriangles', 'findDegenerateFaces', 'findHoleVertIdsByHoleEdges', 'findLeftBoundary', 'findMaxDistanceSq', 'findMaxDistanceSqOneWay', 'findOverhangs', 'findProjection', 'findRightBoundary', 'findSelfCollidingEdges', 'findSelfCollidingTriangles', 'findSelfCollidingTrianglesBS', 'findSignedDistance', 'findUndercuts', 'fixMultipleEdges', 'fixSelfIntersections', 'fixUndercuts', 'floatGridToVdbVolume', 'generalOffsetMesh', 'getAllComponents', 'getAllComponentsVerts', 'getCircumscribedMetric', 'getComplexFillMetric', 'getComponent', 'getComponentVerts', 'getEdgeLengthFillMetric', 'getEdgeLengthStitchMetric', 'getFacesByMinEdgeLength', 'getIncidentFaces', 'getIncidentVerts', 'getInnerFaces', 'getInnerVerts', 'getLargestComponent', 'getLargestComponentVerts', 'getMinAreaMetric', 'getPlaneFillMetric', 'getUniversalMetric', 'gridToMesh', 'hasMultipleEdges', 'identityMetric', 'inflate', 'intersection', 'loadDCMFolder', 'loadDCMsFolder', 'loadDistanceMapFromImage', 'loadLines', 'loadMesh', 'loadPoints', 'loadSceneObject', 'loadTiffDir', 'loadVoxels', 'localFindSelfIntersections', 'localFixSelfIntersections', 'makeBridge', 'makeBridgeEdge', 'makeConvexHull', 'makeCube', 'makeCylinder', 'makeCylinderAdvanced', 'makeDegenerateBandAroundRegion', 'makeMovementBuildBody', 'makeOuterHalfTorus', 'makeSphere', 'makeTorus', 'makeTorusWithComponents', 'makeTorusWithSelfIntersections', 'makeTorusWithSpikes', 'makeTorusWithUndercut', 'makeUVSphere', 'mergeMeshes', 'meshRegularMap', 'meshToDistanceField', 'meshToDistanceVolume', 'meshToLevelSet', 'meshToPointCloud', 'meshToVolume', 'mixed', 'mult', 'offsetMesh', 'planeSectionsToContours2f', 'pointGridSampling', 'pointIterativeSampling', 'pointUniformSampling', 'pointsToMeshFusion', 'positionVertsSmoothly', 'positionVertsSmoothlySharpBd', 'projectAllMeshVertices', 'rayMeshIntersect', 'relax', 'relaxApprox', 'relaxKeepVolume', 'remesh', 'removeSpikes', 'saveAllSlicesToImage', 'saveDistanceMapToImage', 'saveLines', 'saveMesh', 'savePoints', 'saveSceneObject', 'saveSliceToImage', 'saveVoxels', 'segmentByGraphCut', 'shrink', 'simpleVolumeToDenseGrid', 'simpleVolumeToVdbVolume', 'smoothRegionBoundary', 'stitchContours', 'straightenBoundary', 'subdivideMesh', 'suggestVoxelSize', 'surfacePathLength', 'surroundingContour', 'thickenMesh', 'topologyFromTriangles', 'transformed', 'triangulateContours', 'triangulatePointCloud', 'uniteCloseVertices', 'uniteManyMeshes', 'vdbVolumeToSimpleVolume', 'vectorColor', 'vectorConstMeshPtr', 'vectorEdgePath', 'vectorEdges', 'vectorFaceBitSet', 'vectorFaceFace', 'vectorFaces', 'vectorFloat', 'vectorICPPointPair', 'vectorMesh', 'vectorThreeVertIds', 'vectorUndirectedEdgeUndirectedEdge', 'vectorUndirectedEdges', 'vectorVertBitSet', 'vectorVerts', 'voxelBooleanIntersect', 'voxelBooleanSubtract', 'voxelBooleanUnite']

In [32]:
import os
import base64
import json

def get_base64_image(image_path):
    """读取图像并返回其 Base64 编码字符串"""
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def replace_image_data(src_dir, dst_dir):
    """替换 JSON 文件中的 imageData 字段为 dst_dir 中对应 JPEG 文件的 Base64 编码内容"""
    # 确保 dst_dir 存在
    os.makedirs(dst_dir, exist_ok=True)

    for filename in os.listdir(src_dir):
        # 处理 JSON 文件
        if filename.endswith('.json'):
            json_path = os.path.join(src_dir, filename)
            with open(json_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)

            # 获取对应的 JPEG 文件名
            image_name = data['imagePath']
            dst_image_path = os.path.join(dst_dir, image_name)

            # 检查 JPEG 文件是否存在
            if not os.path.exists(dst_image_path):
                print(f"Warning: {dst_image_path} does not exist. Skipping.")
                continue

            # 获取 Base64 编码的 JPEG 内容
            base64_image = get_base64_image(dst_image_path)

            # 替换 JSON 中的 imageData 字段
            data['imageData'] = base64_image

            # 更新 imagePath 字段为 dst_dir 中的路径
            data['imagePath'] = os.path.basename(dst_image_path)

            # 保存修改后的 JSON 文件到 dst_dir
            dst_json_path = os.path.join(dst_dir, filename)
            with open(dst_json_path, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, ensure_ascii=False, indent=4)

            print(f"Processed {filename} and saved to {dst_json_path}")


base_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/04/'

# 定义 src_dir 和 dst_dir
src_dir = os.path.join(base_dir, 'adaptive_threshold_jpg')
dst_dir = os.path.join(base_dir, 'jpg')

# 运行替换函数
replace_image_data(src_dir, dst_dir)

Processed 04_slice_0079.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0079.json
Processed 04_slice_0080.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0080.json
Processed 04_slice_0081.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0081.json
Processed 04_slice_0082.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0082.json
Processed 04_slice_0083.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0083.json
Processed 04_slice_0084.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0084.json
Processed 04_slice_0085.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0085.json
Processed 04_slice_0086.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0086.json
Processed 04_slice_0087.json and saved to D:/Code/CBCT_Reconstruction_ICP/data/04/jpg\04_slice_0087.json
Processed 04_slice_0088.json and saved to D:/Code/CBCT_

In [33]:
import os
import shutil

def copy_json_files(src_dirs, dst_dir):
    """
    将多个源目录中的 JSON 文件复制到目标目录。
    
    参数:
    src_dirs (list): 源目录列表。
    dst_dir (str): 目标目录。
    """
    # 确保目标目录存在
    os.makedirs(dst_dir, exist_ok=True)

    for src_dir in src_dirs:
        # 构建 jpg 子目录路径
        jpg_dir = os.path.join(src_dir, 'jpg')

        if not os.path.exists(jpg_dir):
            print(f"Warning: {jpg_dir} does not exist. Skipping.")
            continue

        # 遍历 jpg 目录中的所有文件
        for filename in os.listdir(jpg_dir):
            if filename.endswith('.json'):
                src_file_path = os.path.join(jpg_dir, filename)
                dst_file_path = os.path.join(dst_dir, filename)

                # 复制 JSON 文件到目标目录
                shutil.copy2(src_file_path, dst_file_path)
                print(f"Copied {src_file_path} to {dst_file_path}")

# 定义源目录
root_dir = 'D:/Code/CBCT_Reconstruction_ICP/data'
sub_dirs = ['01', '02', '03', '04']
src_dirs = [os.path.join(root_dir, sub_dir) for sub_dir in sub_dirs]

# 定义目标目录
dst_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/dataset'

# 运行复制函数
copy_json_files(src_dirs, dst_dir)

Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0100.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0100.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0101.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0101.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0102.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0102.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0103.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0103.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0104.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0104.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0105.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0105.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0106.json to D:/Code/CBCT_Reconstruction_ICP/dataset\01_slice_0106.json
Copied D:/Code/CBCT_Reconstruction_ICP/data\01\jpg\01_slice_0107.json to D:/